# CPSC 330 Lecture 5

#### Lecture plan

- Announcements
- Hyperparameter optimization (25 min)
- Break (5 min)
- Pipelines intro (10 min)
- Pipelines and the Golden Rule (20 min)
- Pipelines and hyperparameter tuning (5 min)



# TODO

add designated question periods

In [1]:
import numpy as np
import pandas as pd
import scipy.stats

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.feature_selection import RFE, RFECV

In [2]:
plt.rcParams['font.size'] = 16

## Announcements

- Midterm grades returned.
- hw5 released last week, deadline extended to **Friday** at 6pm.

## Hyperparameter optimization (25 min)

#### Manual hyperparameter optimization

- We tried this last class.
- Advantage: we may have some intuition about what might work.
- Disadvantage: it takes a lot of work.
- Disadvantage: in very complicated cases, our intuition might be worse than a data-driven approach.

#### Automated hyperparameter optimization

- Advantage: reduce human effort
- Advantage: less prone to error and improve reproducibility
- Advantage: data-driven approaches may be effective
- Disadvantage: may be hard to incorporate intuition
- Disadvantage: be careful about overfitting on the validation set.



There are two automated hyperparameter search methods in scikit-learn:

  - Exhaustive grid search: [`sklearn.model_selection.GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
  - Randomized hyperparameter optimization: [`sklearn.model_selection.RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

In [196]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

#### Exhaustive grid search

- A user specifies a set of values for each hyperparameter. 
- The method considers "product" of the sets and then evaluates each combination one by one.    

Let's remind ourselves how things were going last time:

In [206]:
rf = RandomForestClassifier()

In [207]:
scores = cross_val_score(rf, X_train_transformed, y_train, cv=5);

In [208]:
scores.mean()

0.8541925480371526

Let's start the automated hyperparameter optimization.

In [209]:
param_grid = {
              "n_estimators"     : [10,100],
              "max_depth"        : [3, None],
              "max_features"     : [3, None]
             }
param_grid

{'n_estimators': [10, 100], 'max_depth': [3, None], 'max_features': [3, None]}

- How many combinations in total? 
- $2\times 2\times 2=8$

In [210]:
np.prod(list(map(len, param_grid.values())))

8

In [211]:
rf = RandomForestClassifier(random_state=321)
grid_search = GridSearchCV(rf, param_grid, cv=3, verbose=1)

In [212]:
grid_search.fit(X_train_transformed, y_train);

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:  1.3min finished


In [213]:
grid_search.best_params_

{'max_depth': None, 'max_features': None, 'n_estimators': 100}

- lol... these are the default values.
- I guess they picked good defaults!

In [214]:
grid_search.best_score_

0.8549216369281329

In [215]:
pd.DataFrame(grid_search.cv_results_)[['mean_test_score', 'param_max_depth', 'param_max_features', 'param_n_estimators', 'mean_fit_time', 'rank_test_score']].set_index("rank_test_score").sort_index()

,mean_test_score,param_max_depth,param_max_features,param_n_estimators,mean_fit_time
rank_test_score,,,,,
1,0.854922,None,None,100,14.519656
2,0.848587,None,None,10,1.399292
3,0.848127,None,3,100,1.986127
4,0.844748,3,None,10,0.517570
4,0.844748,3,None,100,5.280527
6,0.839527,None,3,10,0.226066
7,0.761057,3,3,10,0.189320
8,0.760519,3,3,100,0.907236


- Note that the grid search object acts like a scikit-learn model.
- It was actually refit on the _whole_ training set, as discussed earlier in the course!
- I believe it is the same as `grid_search.best_estimator_`.

In [216]:
grid_search.predict(X_test_transformed)

array(['<=50K', '>50K', '<=50K', ..., '>50K', '<=50K', '<=50K'],
      dtype=object)

Problems with exhaustive grid search 

- Required number of models to evaluate grows exponentially with the dimensionally of the configuration space. 
- Exhaustive search may become infeasible fairly quickly. 

Randomized hyperparameter search

- Randomized hyperparameter optimization: [`sklearn.model_selection.RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
- Samples configurations at random until certain budget (e.g., time) is exhausted.
- Advantage: you can choose how many runs you'll do.
- Advantage: you can restrict yourself less on what values you might try.
- Advantage: Adding parameters that do not influence the performance does not affect efficiency.
- Advantage: research shows this is generally a better idea than grid search, see image for intuition:

![](img/randomsearch_bergstra.png)

Source: [Bergstra and Bengio, Random Search for Hyper-Parameter Optimization, JMLR 2012](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf).

- You don't know in advance which hyperparameters are important for your problem.
- But some of them might be unimportant.
- In the left figure, 6 of the 9 searches are useless because they are only varying the unimportant parameter.
- In the right figure, all 9 searches are useful.

Back to syntax. We can have the parameters chosen from a list:

In [217]:
param_choices = {
              "n_estimators"     : [10, 30, 100, 300],
              "max_depth"        : [3, 10, None],
              "max_features"     : [3, 10, None]
             }

You can also give it distributions, instead of lists.

In [218]:
import scipy.stats

In [219]:
param_dist = {
              "n_estimators"     : scipy.stats.randint(low=10, high=300),
              "max_depth"        : scipy.stats.randint(low=10, high=30),
              "max_features"     : scipy.stats.randint(low=10, high=30)
             }

In [220]:
rf = RandomForestClassifier(random_state=321) # Note: you can set other hyperparameters here

In [ ]:
random_search = RandomizedSearchCV(rf, param_distributions = param_dist, 
                                   n_iter = 10, 
                                   cv=3,
                                   verbose=1, random_state=123)

In [ ]:
random_search.fit(X_train_transformed, y_train);

- Note: some hyperparameters significantly affect the training time!
- For example, setting `n_estimators=1000` is going to be very slow.

In [221]:
random_search.best_params_

{'max_depth': 16, 'max_features': 27, 'n_estimators': 93}

- Now we get something different! 
- What's the score?

In [222]:
random_search.best_score_

0.863175750441226

- So, we had 85.4% and now we have 86.1%.
- Is that difference important?
- Do we BELIEVE that difference?
  - We can try it out on the test set.
- But first:  

In [225]:
pd.DataFrame(random_search.cv_results_)[['mean_test_score', 'param_max_depth', 'param_max_features', 'param_n_estimators', 'mean_fit_time', 'rank_test_score']].set_index("rank_test_score").sort_index()

,mean_test_score,param_max_depth,param_max_features,param_n_estimators,mean_fit_time
rank_test_score,,,,,
1,0.863176,16,27,93,4.662635
2,0.862561,20,11,106,2.776636
3,0.861640,23,12,108,3.506636
4,0.861602,17,12,94,2.629149
5,0.861333,14,10,218,5.281985
6,0.860527,27,25,83,3.811981
7,0.860450,25,29,263,15.884677
8,0.860412,10,24,234,8.109985
9,0.859951,25,26,145,7.407122


- Look at the timings, they are quite interesting.
- And now, the test set:

In [223]:
grid_search.score(X_test_transformed, y_test)

0.8527560264087211

In [224]:
random_search.score(X_test_transformed, y_test)

0.8622754491017964

## Fancier methods

- Both `GridSearchCV` and `RandomizedSearchCV` do each trial independently.
- What if you could learn from your experience, e.g. learn that `max_depth=3` is bad?
  - That could save time because you wouldn't try combinations involving `max_depth=3` in the future.
- We can do this with `scikit-optimize`, which is a completely different package from `scikit-learn`
- It uses a technique called "model-based optimization" and we'll specifically use "Bayesian optimization".
  - In short, it uses machine learning to predict what hyperparameters will be good.
  - Machine learning on machine learning!
- As it happens I did my PhD thesis on this topic.

In [311]:
from skopt import BayesSearchCV

- `BayesSearchCV` uses the same interface as `GridSearchCV` and `RandomSearchCV`.
- However, the way we specify the parameter distributions is slightly different.
- Here, we can just give the bounds as tuples.

In [312]:
bayes_opt = BayesSearchCV(
    RandomForestClassifier(random_state=321),
    {
        'n_estimators': (10, 300),  
        'max_depth': (3, 30),
        'max_features': (3, 30)
    },
    n_iter=10,
    cv=3,
    random_state=123,
    verbose=0,
    refit=True
)

In [313]:
%%time
bayes_opt.fit(X_train_transformed, y_train);

CPU times: user 2min 41s, sys: 3.35 s, total: 2min 44s
Wall time: 3min 3s


BayesSearchCV(cv=3, error_score='raise',
              estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                               class_weight=None,
                                               criterion='gini', max_depth=None,
                                               max_features='auto',
                                               max_leaf_nodes=None,
                                               max_samples=None,
                                               min_impurity_decrease=0.0,
                                               min_impurity_split=None,
                                               min_samples_leaf=1,
                                               min_samples_split=2,
                                               min_weight_fraction_leaf=0.0,
                                               n_estimators=100, n_jobs=None,
                                               oob_score=False,
                                 

- It took a similar amount of time to the other methods.
- In reality there is some extra computation to do the "meta-ML".
- However, the overall time is dominated by the time of calling `fit` on the random forests.

In [314]:
bayes_opt.best_params_

{'max_depth': 18, 'max_features': 12, 'n_estimators': 130}

In [315]:
bayes_opt.best_score_

0.8625998157248157

- The score looks promising.
- In theory, it should get even better as we increase `n_iter` (because it has more data to learn from).
- Checking the test score:

In [316]:
bayes_opt.score(X_test_transformed, y_test)

0.8621219100261016

And reproducing the previous test scores for comparison:

In [317]:
random_search.score(X_test_transformed, y_test)

0.8622754491017964

In [318]:
grid_search.score(X_test_transformed, y_test)

0.8527560264087211

- In this case, it seems we weren't overfitting on the validation set and this exercise was actually useful.
- Should I always use this? Not necessarily.
- Disadvantage: requires installation.
- Disadvantage: when number of trials is large (e.g. hundreds), the meta-ML can actually get too slow.
- Disadvantage: harder parallelize the search because each trial depends on the previous ones.
  - Note `n_jobs` parameter for `GridSearchCV` and `RandomizedSearchCV`.  
  - `BayesSearchCV` also has this parameter.
  - It can definitely parallelize the folds.
  - The search will be less effective if it parallelizes further.

- Can I generalize this to say `BayesSearchCV` > `RandomizedSearchCV` > `GridSearchCV`?
- Not quite. I'd say `RandomizedSearchCV` > `GridSearchCV` is pretty reasonable
- But we should think a bit more carefully about `BayesSearchCV` for the above reasons.
- `RandomizedSearchCV` is often a reasonable choice.

## Break (5 min)

## Pipelines intro (10 min)

Returning to our house price prediction dataset from last week:

In [4]:
df = pd.read_csv("data/week_05_house-prices/train.csv")

In [5]:
df_trainvalid, df_test = train_test_split(df, test_size=0.1, random_state=123)

In [6]:
df_train, df_valid = train_test_split(df_trainvalid, test_size=0.2, random_state=123)

In [7]:
target               = ['SalePrice']
drop_features        = ['Id']
numeric_features     = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 
                        'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                        'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 
                        'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 
                        'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 
                        'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 
                        'ScreenPorch', 'PoolArea', 'MiscVal', 'YrSold']
ordinal_features_reg = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 
                        'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']
ordinal_features_oth = ['BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 
                        'Functional',  'Fence']
categorical_features = list(set(df_train.columns) - set(target) - set(drop_features) - 
                            set(numeric_features) - 
                            set(ordinal_features_reg) - set(ordinal_features_oth))
all_features = numeric_features + ordinal_features_reg + categorical_features + ordinal_features_oth

ordering = ['Po', 'Fa', 'TA', 'Gd', 'Ex'] # if N/A it will just impute something, per below

In [8]:
y_train = df_train["SalePrice"]
y_valid = df_valid["SalePrice"]
y_trainvalid = df_trainvalid["SalePrice"]
y_test  = df_test["SalePrice"]

In [9]:
y_train_log = np.log(y_train)
y_valid_log = np.log(y_valid)
y_trainvalid_log = np.log(y_trainvalid)
y_test_log  = np.log(y_test)

What are the next steps?

- Feature transformations/encodings.
- What if we have many?
- These can be combined into a scikit-learn [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

In [10]:
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [11]:
ordinal_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(categories=[ordering for i in ordinal_features_reg]))
])

In [12]:
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='?')),
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

The `ColumnTransformer` now combines these `Pipeline` objects instead of the individual transformers:

In [13]:
preprocessor = ColumnTransformer([
    ('numeric', numeric_transformer, numeric_features),
    ('ordinal', ordinal_transformer, ordinal_features_reg),
    ('categorical', categorical_transformer, categorical_features + ordinal_features_oth)
])

This next line fits all 6 transformers:

In [14]:
preprocessor.fit(df_train);

In [15]:
ohe = preprocessor.named_transformers_['categorical'].named_steps['onehot']
ohe_feature_names = list(ohe.get_feature_names(categorical_features + ordinal_features_oth))
new_columns = numeric_features + ordinal_features_reg + ohe_feature_names
new_columns;

In [16]:
X_train_enc = pd.DataFrame(preprocessor.transform(df_train), index=df_train.index, columns=new_columns)
X_valid_enc = pd.DataFrame(preprocessor.transform(df_valid), index=df_valid.index, columns=new_columns)
X_test_enc  = pd.DataFrame(preprocessor.transform(df_test),  index=df_test.index,  columns=new_columns)

In [17]:
X_train_enc.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Typ,Fence_?,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw
1341,-0.165978,0.338699,-0.062802,-0.505587,1.048406,0.927764,-0.577947,0.791412,-0.284437,-0.622612,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
459,-0.026237,-0.354633,-0.764983,-1.415471,-0.731683,-1.679272,0.294327,-0.544487,-0.284437,-0.120404,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
367,1.464330,-0.133037,-0.062802,-0.505587,-0.328644,-1.099931,1.074497,-0.149451,-0.284437,0.336963,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
894,-0.259139,-0.354322,-0.764983,-0.505587,0.242328,-0.279197,-0.577947,-0.937398,-0.284437,-1.295212,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
672,-0.026237,0.084927,-0.062802,0.404297,0.175155,-0.375754,-0.577947,0.691592,-0.284437,-0.306490,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


- So far, we just accomplished the same thing with cleaner code.
  - Helps avoid messing up the order of the features.
- Next, we will use this to accomplish something new.
- But first...

## Pipelines and the Golden Rule (20 min)

I did something like this in the hw4 solution (note I'm just using the `model` here, not the `pipeline`):

In [18]:
X_trainvalid_enc = pd.concat((X_train_enc, X_valid_enc))

In [20]:
model = Ridge(alpha=100)

In [21]:
cross_val_score(model, X_trainvalid_enc, y_trainvalid_log, cv=10)

array([-0.05559473, -0.04102244, -0.05763048, -0.0949194 , -0.07345046,
       -0.06285405, -0.05423519, -0.02718618, -0.01933039, -0.06156132])

- What is the problem here?
  - Discuss for 2-3 minutes.
  - Don't look ahead!

<br><br><br><br><br><br><br><br>

- The folds are a random reshuffling.
  - Golden Rule: don't call `fit` on your validation data.
  - But in my cross-validation splits, the validation portion will contain some of the original training data, which was used to `fit` the preprocessor.
  - Thus, the cross-validation training data was preprocessed "knowing something" about the cross-validation validation data.
     - So the cross-validation validation data is not truly "unseen data".
     - This is a "leak".
  - This is _extremely confusing_ because we have the original train/validation and the cross-validation train/validation.
  - Draw a picture here!
- I accidentally violated the Golden Rule in hw4 exercise 4(c) doing this with `RandomizedSearchCV`.
  - Let's take a look there for a minute.
  - I'll update the solutions.

Ok, so I'll only use the train split - which I also suggested in the hw4 solution. Does this solve the problem?
  - Discuss for 2-3 minutes.
  - Don't look ahead!

In [22]:
cross_val_score(model, X_train_enc, y_train_log, cv=10)

array([0.82163027, 0.94802539, 0.88206276, 0.91522733, 0.56098685,
       0.84892397, 0.86539653, 0.89571204, 0.90336091, 0.9062442 ])

<br><br><br><br><br><br>

- No! I still have folds where the train split was preprocessed based on data in the validation split.
- Ok, so I'll fit the preprocessing on the combined "trainvalid" split - does that solve the problem?

In [23]:
preprocessor2 = ColumnTransformer([
    ('numeric', numeric_transformer, numeric_features),
    ('ordinal', ordinal_transformer, ordinal_features_reg),
    ('categorical', categorical_transformer, categorical_features + ordinal_features_oth)
])

In [24]:
X_trainvalid_enc2 = preprocessor2.fit_transform(df_trainvalid);

In [25]:
cross_val_score(model, X_trainvalid_enc2, y_trainvalid_log, cv=10)

array([0.91330975, 0.86542043, 0.84417442, 0.87904043, 0.90593103,
       0.89385594, 0.86962887, 0.90348459, 0.5922736 , 0.91025991])

<br><br><br><br><br><br>

- No! Same problem again!
- What if we use a `Pipeline`?
  - So far we just used a `Pipeline` for the preprocessing.
  - This time we'll combine **the preprocessing and the model** with a `Pipeline` as well.

In [26]:
model = Ridge(alpha=100)

In [27]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)])

The next line fits the transformer **and** the model; there was no need to call `preprocessor.fit()` before this.

In [28]:
pipeline.fit(df_train, y_train_log);

Note how I passed in `df_train`, not `df_train_enc`.

In [29]:
pipeline.predict(df_train)

array([11.97330353, 11.69417912, 11.9050535 , ..., 12.20638141,
       11.99713426, 11.81988184])

In [30]:
pipeline.score(df_train, y_train_log)

0.8963159928298279

In [31]:
pipeline.score(df_valid, y_valid_log)

0.8913218700485418

We'll use this `Pipeline` on the combined train and validation sets:

In [33]:
cross_val_score(pipeline, df_trainvalid, y_trainvalid_log, cv=10)

array([0.91329227, 0.86442022, 0.84424495, 0.87895509, 0.90585488,
       0.89379337, 0.86963715, 0.90347339, 0.58863687, 0.91024683])

- Does this solve the problem?
  - Discuss for 2-3 minutes.
  - Don't look ahead!
  
<br><br><br><br><br><br>

Yes! Why does this work?

- Because `cross_val_score` calls `fit` for each fold.
- And this includes fitting the preprocessor.
- Thus, there is actually no difference between `df_train` and `df_valid` - nothing has actually been done to them yet!
  - `df_trainvalid` is just the part that's not the test set, that's all.

(optional note) Yet another idea could be to do the cross-validatin using only the validation split. I believe this does not technically violate the Golden Rule, but it's very wasteful in terms of data and is also not really representative of how your model will eventually be trained. The `Pipeline` approach is much better. 

## Pipelines and hyperparameter optimization (5 min)

- The same problems arise when doing hyperparameter optimization, simply because these methods do cross-validation.
- We can avoid them with a `Pipeline` in the same way.
- I'll optimize `alpha`, so I'll create a new pipeline where `alpha` is not specified:

In [34]:
pipeline = Pipeline([('preprocessor', preprocessor),
                      ('model', Ridge())])

param_grid = {
    'preprocessor__numeric__imputer__strategy': ['mean', 'median'],
    'model__alpha': [1.0, 10, 100],
}

- Above: we have a nesting of transformers. 
- We can access the parameters of the "inner" objects by using `__` to go "deeper":
  - `model__alpha`: "the `alpha` of the model (of the pipeline)"
  - `preprocessor__numeric__imputer__strategy`: "the strategy of the imputer of the numeric transformer of the preprocessor (of the pipeline)"

In [35]:
grid_search = GridSearchCV(pipeline, param_grid, cv=10)
grid_search.fit(df_trainvalid, y_trainvalid_log);

In [36]:
grid_search.best_params_

{'model__alpha': 10, 'preprocessor__numeric__imputer__strategy': 'median'}

This is particularly useful when there are serious hyperparameters in the preprocessing pipeline, e.g. if you're using `CountVectorizer`.